In [1]:
import torch
import numpy as np

from net_i2l import NetIntel


nets = {}
#nets_names = ('resnet-18', 'alexnet', 'vgg', 'densenet', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7')
nets_names = ('mobilenet_v3_large',)
for net_name in nets_names:
    print(f'Processing net with embedding {net_name}')
    nets[net_name] = NetIntel(embedding=net_name)
    print(f'Mean embedding time: {np.mean(nets[net_name].stats["emb_times"])} += {np.std(nets[net_name].stats["emb_times"])}')

def eval(net, net_name):
    print(f'Evaluation of net {net_name}')

    net.model.eval()
    loss_list = []
    oks = []
    wrongs = []
    n_correct = 0
    n_fail = 0
    for x, y_true, sample, label in net.data.loader(group='test', batch_size=1):
        
        y_pred = net.model(x)
        loss_list.append(net.trainer.criterion(y_pred, y_true).data)
        
        target_pred = torch.argmax(y_pred).item()
        if target_pred == y_true[0].item():
            n_correct += 1
            oks.append((sample[0], label[0]))
        else:
            n_fail += 1
            wrongs.append((sample[0], label[0], net.data.target2label[target_pred]))

    acc = n_correct / (n_correct + n_fail)
    loss = np.mean([l.item() for l in loss_list])


    print(f'Loss: {loss}, Acc: {acc}')

/Users/kitt/miniconda3/envs/trans/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processing net with embedding mobilenet_v3_large
# Intel train samples: 300
# Intel dev samples: 120
# Intel test samples: 1800
Data initialized: n = 1280, m = 6, pt = 300, pk = 1800, labels: {0: 'buildings', 1: 'forest', 2: 'glacier', 3: 'mountain', 4: 'sea', 5: 'street'}
Mean embedding time: 0.0358343007328274 += 0.009211131600370629


In [17]:
from net_i2l import I2LModel

# Reinit
for net_name, net in nets.items():
    net.model = I2LModel(net.data.n, net.hidden, net.data.m)
    net.trainer.reinit_optimizer()

In [2]:
for net_name, net in nets.items():
    print(f'\n\n == Learning net {net_name}')
    net.learn(epochs=1500, patience=10)
    print(f'= last epoch: {net.stats["last_epoch"]}')
    print(f'= train time: {net.stats["train_time"]}')
    print(f'= train loss: {net.stats["train_loss_list"][-1]}')
    print(f'= dev loss: {net.stats["val_loss_list"][-1]}')
    eval(net, net_name)



 == Learning net mobilenet_v3_large
Early stopping, dev_loss tail: [0.22011072747409344, 0.22553562000393867, 0.21926244953647256, 0.2381633184850216, 0.2414576644077897, 0.21964311716146767, 0.22042589401826262, 0.22075440594926476, 0.24166752956807613]
Final train loss: 0.0385642026207949, dev loss: 0.2766927513293922
= last epoch: 34
= train time: 0.8004870414733887
= train loss: 0.0385642026207949
= dev loss: 0.2766927513293922
Evaluation of net mobilenet_v3_large
Loss: 0.2588300538226031, Acc: 0.915


In [44]:
from urllib.request import urlopen
from PIL import Image
import timm
from timm.data.config import resolve_data_config
print([m for m in timm.list_models() if 'mobilenetv3_large' in m])

img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model(
    'mobilenetv3_large_100_miil_in21k',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = resolve_data_config(args={}, model=model)
transforms = timm.data.create_transform(**data_config, is_training=False)

#output = model(transforms(img).unsqueeze(0))  # output is (batch_size, num_features) shaped tensor

# or equivalently (without needing to set num_classes=0)

output = model.forward_features(transforms(img).unsqueeze(0))
# output is unpooled, a (1, 960, 7, 7) shaped tensor

output = model.forward_head(output, pre_logits=True)
# output is a (1, num_features) shaped tensor

print(output.detach().flatten().numpy().shape)

['mobilenetv3_large_075', 'mobilenetv3_large_100', 'mobilenetv3_large_100_miil', 'mobilenetv3_large_100_miil_in21k', 'tf_mobilenetv3_large_075', 'tf_mobilenetv3_large_100', 'tf_mobilenetv3_large_minimal_100']
(1280,)
